In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import time
import os
import sqlite3
from datetime import datetime, timedelta 

# ==========================================
# 1. DB 관리 클래스 (로그/매매내역 저장)
# ==========================================
class BotDatabase:
    def __init__(self, db_name=r"C:\dev\stock-ml-project\trading.db"):
        self.db_name = db_name
        self.init_db()
        
    def init_db(self):
        conn = sqlite3.connect(self.db_name)
        c = conn.cursor()
        # 매매 내역 테이블
        c.execute('''CREATE TABLE IF NOT EXISTS trades
                     (id INTEGER PRIMARY KEY AUTOINCREMENT,
                      time TEXT, symbol TEXT, type TEXT, 
                      price REAL, qty INTEGER, profit REAL)''')
        # 로그 테이블
        c.execute('''CREATE TABLE IF NOT EXISTS logs
                     (id INTEGER PRIMARY KEY AUTOINCREMENT,
                      time TEXT, message TEXT)''')
        conn.commit()
        conn.close()
        
    def log(self, message):
        try:
            conn = sqlite3.connect(self.db_name)
            c = conn.cursor()
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            c.execute("INSERT INTO logs (time, message) VALUES (?, ?)", (now, message))
            conn.commit()
            conn.close()
            print(f"[{now}] {message}") # 콘솔에도 출력
        except: pass

    def save_trade(self, symbol, type, price, qty, profit=0):
        try:
            conn = sqlite3.connect(self.db_name)
            c = conn.cursor()
            now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            c.execute("INSERT INTO trades (time, symbol, type, price, qty, profit) VALUES (?, ?, ?, ?, ?, ?)",
                      (now, symbol, type, price, qty, profit))
            conn.commit()
            conn.close()
        except: pass

# ==========================================
# 2. KIS API 클래스
# ==========================================
class KisDataFetcher:
    def __init__(self, app_key, app_secret, acc_no, mode="real"):
        self.app_key = app_key
        self.app_secret = app_secret
        
        # 계좌번호 하이픈 유무 자동 처리
        if '-' in acc_no:
            self.acc_no_prefix = acc_no.split('-')[0]
            self.acc_no_suffix = acc_no.split('-')[1]
        else:
            self.acc_no_prefix = acc_no[:8]
            self.acc_no_suffix = acc_no[8:]
            
        self.mode = mode
        
        if mode == "real":
            self.base_url = "https://openapi.koreainvestment.com:9443"
            self.tr_id_kr_buy = "TTTC0802U"
            self.tr_id_kr_sell = "TTTC0801U"
            self.tr_id_us_buy = "JTTT1002U" 
            self.tr_id_us_sell = "JTTT1006U" 
        else:
            self.base_url = "https://openapivts.koreainvestment.com:29443"
            self.tr_id_kr_buy = "VTTC0802U"
            self.tr_id_kr_sell = "VTTC0801U"
            self.tr_id_us_buy = "VTTT1002U"
            self.tr_id_us_sell = "VTTT1006U"
            
        self.access_token = None
        self.token_file = "kis_token_cache.json"

    def auth(self, force=False):
        # force=True 이면 무조건 새로 발급
        if not force and os.path.exists(self.token_file):
            try:
                with open(self.token_file, 'r') as f:
                    saved_data = json.load(f)
                saved_time = datetime.strptime(saved_data['timestamp'], "%Y-%m-%d %H:%M:%S")
                # 🔥 토큰 유효시간: 1시간 → 안전하게 50분으로 잡자
                if datetime.now() - saved_time < timedelta(minutes=50):
                    self.access_token = saved_data['access_token']
                    return
            except:
                pass

        print("🔑 토큰 재발급...")
        url = f"{self.base_url}/oauth2/tokenP"
        headers = {"content-type": "application/json"}
        body = {
            "grant_type": "client_credentials",
            "appkey": self.app_key,
            "appsecret": self.app_secret
        }
        res = requests.post(url, headers=headers, data=json.dumps(body))
        if res.status_code == 200:
            self.access_token = res.json()["access_token"]
            with open(self.token_file, 'w') as f:
                json.dump({
                    "access_token": self.access_token,
                    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                }, f)
        else:
            raise Exception(f"토큰 발급 실패: {res.text}")
        
    def _request(self, method, url, tr_id, **kwargs):
        headers = self.get_headers(tr_id)
        res = requests.request(method, url, headers=headers, **kwargs)

        # 토큰 만료 체크
        try:
            data = res.json()
        except:
            data = {}

        if res.status_code == 500 and data.get("msg_cd") == "EGW00123":
            # 기간이 만료된 token 입니다.
            print("♻️ 토큰 만료 감지 → 재발급 후 재시도")
            self.auth(force=True)
            headers = self.get_headers(tr_id)
            res = requests.request(method, url, headers=headers, **kwargs)

        return res
    def get_headers(self, tr_id):
        if not self.access_token:
            self.auth()
        return {
            "content-type": "application/json",
            "authorization": f"Bearer {self.access_token}",
            "appkey": self.app_key,
            "appsecret": self.app_secret,
            "tr_id": tr_id,
            "custtype": "P",   # ✅ 개인 계정이면 무조건 P
        }

    def get_kr_buyable_cash(self):
        url = f"{self.base_url}/uapi/domestic-stock/v1/trading/inquire-psbl-order"
        tr_id = "VTTC8908R" if self.mode == "virtual" else "TTTC8908R"
        headers = self.get_headers(tr_id)
        params = {
            "CANO": self.acc_no_prefix,
            "ACNT_PRDT_CD": self.acc_no_suffix,
            "PDNO": "005930",
            "ORD_UNPR": "0",
            "ORD_DVSN": "01",
            "CMA_EVLU_AMT_ICLD_YN": "Y",
            "OVRS_ICLD_YN": "N"
        }
        res = requests.get(url, headers=headers, params=params)

        try:
            data = res.json()
        except Exception as e:
            print("❌ get_kr_buyable_cash JSON 파싱 실패:", e, res.text)
            return 0

        if res.status_code != 200 or data.get('rt_cd') != '0':
            print("❌ get_kr_buyable_cash 실패:", res.status_code, data.get('rt_cd'), data.get('msg1'))
            return 0

        return int(data['output']['ord_psbl_cash'])


    def get_us_buyable_cash(self):
        url = f"{self.base_url}/uapi/overseas-stock/v1/trading/inquire-balance"
        tr_id = "VTTS3012R" if self.mode == "virtual" else "TTTS3012R"
        headers = self.get_headers(tr_id)
        params = {
            "CANO": self.acc_no_prefix,
            "ACNT_PRDT_CD": self.acc_no_suffix,
            "OVRS_EXCG_CD": "NASD",
            "TR_CRCY_CD": "USD",
            "CTX_AREA_FK200": "",
            "CTX_AREA_NK200": ""
        }
        res = requests.get(url, headers=headers, params=params)

        try:
            data = res.json()
        except Exception as e:
            print("❌ get_us_buyable_cash JSON 파싱 실패:", e, res.text)
            return 0.0

        if res.status_code != 200 or data.get('rt_cd') != '0':
            print("❌ get_us_buyable_cash 실패:", res.status_code, data.get('rt_cd'), data.get('msg1'))
            return 0.0

        # 해외 예수금은 output2 / output3 등 위치가 계좌 설정에 따라 달라질 수 있음
        try:
            return float(data['output2']['ovrs_ord_psbl_amt'])
        except Exception as e:
            print("⚠️ get_us_buyable_cash 금액 파싱 실패:", e, data)
            return 0.0


    def get_kr_balance(self):
        url = f"{self.base_url}/uapi/domestic-stock/v1/trading/inquire-balance"
        tr_id = "VTTC8434R" if self.mode == "virtual" else "TTTC8434R"

        res = self._request("GET", url, tr_id, params={
            "CANO": self.acc_no_prefix,
            "ACNT_PRDT_CD": self.acc_no_suffix,
            "AFHR_FLPR_YN": "N",
            "OFL_YN": "N",
            "INQR_DVSN": "02",
            "UNPR_DVSN": "01",
            "FUND_STTL_ICLD_YN": "N",
            "FNCG_AMT_AUTO_RDPT_YN": "N",
            "PRCS_DVSN": "01",
            "CTX_AREA_FK100": "",
            "CTX_AREA_NK100": ""
        })

        stock_dict = {}
        try:
            data = res.json()
        except Exception as e:
            print("❌ get_kr_balance JSON 파싱 실패:", e, res.text)
            return stock_dict

        if res.status_code != 200:
            print("❌ get_kr_balance HTTP 에러:", res.status_code, res.text)
            return stock_dict

        if data.get('rt_cd') != '0':
            print("❌ get_kr_balance rt_cd 실패:", data.get('rt_cd'), data.get('msg1'))
            return stock_dict

        for item in data.get('output1', []):
            try:
                if int(item['hldg_qty']) > 0:
                    stock_dict[item['pdno']] = {
                        'qty': int(item['hldg_qty']),
                        'avg_price': float(item['pchs_avg_pric'])
                    }
            except Exception as e:
                print("⚠️ get_kr_balance 개별 항목 파싱 오류:", e, item)

        return stock_dict


    def send_kr_order(self, symbol, order_type, qty):
        url = f"{self.base_url}/uapi/domestic-stock/v1/trading/order-cash"
        tr_id = self.tr_id_kr_buy if order_type == "buy" else self.tr_id_kr_sell
        headers = self.get_headers(tr_id)
        data = {
            "CANO": self.acc_no_prefix, "ACNT_PRDT_CD": self.acc_no_suffix,
            "PDNO": symbol, "ORD_DVSN": "01", "ORD_QTY": str(qty), "ORD_UNPR": "0"
        }
        res = requests.post(url, headers=headers, data=json.dumps(data))
        if res.status_code == 200 and res.json()['rt_cd'] == '0':
            return True
        return False

    def get_kr_current_price(self, symbol):
        url = f"{self.base_url}/uapi/domestic-stock/v1/quotations/inquire-price"
        headers = self.get_headers("FHKST01010100")
        params = {"FID_COND_MRKT_DIV_CODE": "J", "FID_INPUT_ISCD": symbol}
        res = requests.get(url, headers=headers, params=params)

        try:
            data = res.json()
        except Exception as e:
            print("❌ get_kr_current_price JSON 실패:", e, res.text)
            return None

        if res.status_code != 200 or data.get('rt_cd') != '0':
            print("❌ get_kr_current_price 실패:", res.status_code, data.get('rt_cd'), data.get('msg1'), symbol)
            return None

        return float(data['output']['stck_prpr'])


    def get_us_current_price(self, exchange, symbol):
        url = f"{self.base_url}/uapi/overseas-price/v1/quotations/price"
        tr_id = "HHDFS76200200"
        res = self._request("GET", url, tr_id, params={
            "AUTH": "",
            "EXCD": exchange,
            "SYMB": symbol
        })

        try:
            data = res.json()
        except Exception as e:
            print("❌ get_us_current_price JSON 실패:", e, res.text)
            return None

        if res.status_code != 200 or data.get('rt_cd') != '0':
            print("❌ get_us_current_price 실패:", res.status_code, data.get('rt_cd'), data.get('msg1'), exchange, symbol)
            return None

        return float(data['output']['last'])

    def get_us_balance(self):
        url = f"{self.base_url}/uapi/overseas-stock/v1/trading/inquire-balance"
        tr_id = "VTTS3012R" if self.mode == "virtual" else "TTTS3012R"
        headers = self.get_headers(tr_id)
        params = {
            "CANO": self.acc_no_prefix,
            "ACNT_PRDT_CD": self.acc_no_suffix,
            "OVRS_EXCG_CD": "NASD",
            "TR_CRCY_CD": "USD",
            "CTX_AREA_FK200": "",
            "CTX_AREA_NK200": ""
        }
        res = requests.get(url, headers=headers, params=params)

        stock_dict = {}
        try:
            data = res.json()
        except Exception as e:
            print("❌ get_us_balance JSON 파싱 실패:", e, res.text)
            return stock_dict

        if res.status_code != 200:
            print("❌ get_us_balance HTTP 에러:", res.status_code, res.text)
            return stock_dict

        if data.get('rt_cd') != '0':
            print("❌ get_us_balance rt_cd 실패:", data.get('rt_cd'), data.get('msg1'))
            return stock_dict

        for item in data.get('output1', []):
            try:
                qty = float(item['ovrs_cblc_qty'])
                if qty > 0:
                    stock_dict[item['ovrs_pdno']] = {
                        'qty': qty,
                        'avg_price': float(item['pchs_avg_pric'])
                    }
            except Exception as e:
                print("⚠️ get_us_balance 개별 항목 파싱 오류:", e, item)

        return stock_dict


    def send_us_order(self, exchange, symbol, order_type, qty, price):
        url = f"{self.base_url}/uapi/overseas-stock/v1/trading/order"
        tr_id = self.tr_id_us_buy if order_type == "buy" else self.tr_id_us_sell
        headers = self.get_headers(tr_id)
        data = {
            "CANO": self.acc_no_prefix, "ACNT_PRDT_CD": self.acc_no_suffix,
            "OVRS_EXCG_CD": exchange, "PDNO": symbol,
            "ORD_QTY": str(qty), "ORD_UNPR": str(price), "ORD_DVSN": "00", "ORD_SVR_DVSN_CD": "0" 
        }
        res = requests.post(url, headers=headers, data=json.dumps(data))
        if res.status_code == 200 and res.json()['rt_cd'] == '0':
            return True
        return False
    
    def get_minute_ohlcv_5m(self, symbol, count=200):
        """
        국내 5분봉 OHLCV 조회
        """
        if not self.access_token:
            self.auth()

        url = f"{self.base_url}/uapi/domestic-stock/v1/quotations/inquire-time-itemchartprice"
        tr_id = "FHKST03010200"   # 국내 분봉 차트 TR (실전용)

        headers = self.get_headers(tr_id)
        params = {
            "FID_ETC_CLS_CODE": "",
            "FID_COND_MRKT_DIV_CODE": "J",   # 주식
            "FID_INPUT_ISCD": symbol,        # 종목코드
            "FID_INPUT_HOUR_1": "",          # 기준시간 ("" = 최신부터)
            "FID_PW_DATA_INCU_YN": "N",
            "FID_TIME_INTERVAL": "5"         # ✅ 5분봉
        }

        res = requests.get(url, headers=headers, params=params)

        try:
            data = res.json()
        except Exception as e:
            print("❌ get_minute_ohlcv_5m JSON 실패:", e, res.text)
            return pd.DataFrame()

        if res.status_code != 200 or data.get("rt_cd") != "0":
            print("❌ get_minute_ohlcv_5m 실패:", res.status_code, data.get("rt_cd"), data.get("msg1"), symbol)
            return pd.DataFrame()

        if "output2" not in data or not data["output2"]:
            print("❌ get_minute_ohlcv_5m output2 비어있음:", symbol)
            return pd.DataFrame()

        df = pd.DataFrame(data["output2"])

        # KIS 분봉 필드명 기준 (시간/시가/고가/저가/종가/거래량)
        # 실제 필드명이 다르면 여기만 한 번 print(df.columns) 찍어서 맞춰주면 됨
        # 예시: ['stck_bsop_date', 'stck_cntg_hour', 'stck_oprc', 'stck_hgpr', 'stck_lwpr', 'stck_prpr', 'acml_vol']
        try:
            df = df[[
                "stck_bsop_date",
                "stck_cntg_hour",
                "stck_oprc",
                "stck_hgpr",
                "stck_lwpr",
                "stck_prpr",
                "cntg_vol"          # ✅ 여기!
            ]]
        except KeyError:
            print("⚠️ get_minute_ohlcv_5m 컬럼 매핑 실패, 실제 컬럼:", df.columns.tolist())
            return pd.DataFrame()

        # 날짜 + 시간 → datetime index
        df["datetime"] = pd.to_datetime(df["stck_bsop_date"] + df["stck_cntg_hour"])
        df.set_index("datetime", inplace=True)

        df = df.rename(columns={
            "stck_oprc": "open",
            "stck_hgpr": "high",
            "stck_lwpr": "low",
            "stck_prpr": "close",
            "cntg_vol": "volume"    # ✅ 여기!
        })

        df = df[["open", "high", "low", "close", "volume"]].astype(float)

        # API가 과거→현재 역순으로 줄 수도 있어서 정렬
        df = df.sort_index()

        # 최근 N개만 사용
        if len(df) > count:
            df = df.iloc[-count:]

        return df

    def get_ohlcv(self, region, symbol, exchange=None):
        if region == "KR":
            # ✅ 여기서 5분봉 함수로 변경
            return self.get_minute_ohlcv_5m(symbol)
        else:
            return self.get_overseas_daily_ohlcv(exchange, symbol)

    def get_daily_ohlcv(self, symbol):
        if not self.access_token: self.auth()
        url = f"{self.base_url}/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice"
        headers = self.get_headers("FHKST03010100")
        params = {
            "FID_COND_MRKT_DIV_CODE": "J",
            "FID_INPUT_ISCD": symbol,
            "FID_INPUT_DATE_1": (datetime.now() - timedelta(days=100)).strftime("%Y%m%d"),
            "FID_INPUT_DATE_2": datetime.now().strftime("%Y%m%d"),
            "FID_PERIOD_DIV_CODE": "D",
            "FID_ORG_ADJ_PRC": "1"
        }
        res = requests.get(url, headers=headers, params=params)

        try:
            data = res.json()
        except Exception as e:
            print("❌ get_daily_ohlcv JSON 실패:", e, res.text)
            return pd.DataFrame()

        if res.status_code != 200 or data.get('rt_cd') != '0':
            print("❌ get_daily_ohlcv 실패:", res.status_code, data.get('rt_cd'), data.get('msg1'), symbol)
            return pd.DataFrame()

        if 'output2' not in data or not data['output2']:
            print("❌ get_daily_ohlcv output2 비어있음:", symbol)
            return pd.DataFrame()

        df = pd.DataFrame(data['output2'])
        df = df[['stck_bsop_date', 'stck_oprc', 'stck_hgpr', 'stck_lwpr', 'stck_clpr', 'acml_vol']]
        df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
        return df.astype(float).sort_index()


    def get_overseas_daily_ohlcv(self, exchange, symbol):
        if not self.access_token: self.auth()
        url = f"{self.base_url}/uapi/overseas-price/v1/quotations/dailyprice"
        headers = self.get_headers("HHDFS76240000")
        params = {"AUTH": "", "EXCD": exchange, "SYMB": symbol, "GUBN": "0", "BYMD": datetime.now().strftime("%Y%m%d"), "MODP": "1"}
        res = requests.get(url, headers=headers, params=params)
        if res.status_code == 200 and res.json()['rt_cd'] == '0':
            df = pd.DataFrame(res.json()['output2'])
            df = df[['xymd', 'open', 'high', 'low', 'clos', 'tvol']]
            df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
            df['date'] = pd.to_datetime(df['date']); df.set_index('date', inplace=True)
            return df.astype(float).sort_index()
        return pd.DataFrame()

# ==========================================
# 3. 통합 실시간 트레이더 (DB 연동 + 리포트)
# ==========================================
class GlobalRealTimeTrader:
    def __init__(self, fetcher, targets, params, db): # [수정] db 인자 추가
        self.fetcher = fetcher
        self.targets = targets
        self.params = params
        self.trade_state = {} 
        self.db = db # [수정] db 객체 연결

    def is_market_open(self, region):
        now = datetime.now()
        if region == "KR":
            return (now.hour == 9) or (now.hour > 9 and now.hour < 15) or (now.hour == 15 and now.minute <= 20)
        else:
            return (now.hour >= 21) or (now.hour < 7)

    def run_check(self):
        # [수정] 화면 출력 대신 DB에 로그 저장
        self.db.log(f"⏰ 감시 시작 (보유 종목 스캔 중...)")
        
        # 1. 잔고 및 예수금 조회
        try:
            kr_balance = self.fetcher.get_kr_balance()
            us_balance = self.fetcher.get_us_balance()
            cash_krw = self.fetcher.get_kr_buyable_cash()
            cash_usd = self.fetcher.get_us_buyable_cash()
        except Exception as e:
            self.db.log(f"⚠️ 잔고 조회 실패: {e}")
            return

        total_holdings_count = len(kr_balance) + len(us_balance)
        self.db.log(f"💰 상태: 보유 {total_holdings_count}종목 | KRW: {cash_krw:,}원 / USD: ${cash_usd:,.2f}")
        
        count_checked = 0
        count_satisfied = 0
        
        # 2. 종목 순회
        for target in self.targets:
            region = target['region']
            symbol = target['symbol']
            excd = target.get('excd', None)
                    
            time.sleep(0.2) 
                    
            if not self.is_market_open(region):
                # US는 장 외라 스킵되는게 정상, KR인데 스킵되면 그때 보자
                # self.db.log(f"⏳ 시장 미개장 스킵: {region} {symbol}")
                continue
                    
            try:
                current_price = 0
                has_stock = False
                my_info = None
                        
                if region == "KR":
                    current_price = self.fetcher.get_kr_current_price(symbol)
                    if not current_price:
                        self.db.log(f"❌ 현재가 조회 실패: {symbol}")
                        continue

                    if symbol in kr_balance:
                        has_stock = True
                        my_info = kr_balance[symbol]
                else:
                    current_price = self.fetcher.get_us_current_price(excd, symbol)
                    if not current_price:
                        # US 쪽은 아직 장도 닫혀있으니 당장은 참고용
                        self.db.log(f"❌ US 현재가 조회 실패: {symbol}")
                        continue

                    if symbol in us_balance:
                        has_stock = True
                        my_info = us_balance[symbol]
                        
                df = self.fetcher.get_ohlcv(region, symbol, excd)
                if df.empty:
                    self.db.log(f"❌ 차트 데이터 없음(빈 df): {region} {symbol}")
                    continue

                # 여기까지 왔으면 “실제로 분석한 종목 1개”
                count_checked += 1

                df['support'] = df['low'].rolling(self.params['lookback']).min()
                df['at_support'] = df['low'] <= df['support'] * (1 + self.params['band_pct'])
                df['is_bullish'] = df['close'] > df['open']
                df['price_up'] = df['close'] > df['close'].shift(1)
                        
                entry_signal = (df.iloc[-1]['at_support'] and df.iloc[-1]['is_bullish'] and df.iloc[-1]['price_up'])
                        
                if entry_signal:
                    count_satisfied += 1
                
                # --- [매수 로직] ---
                if entry_signal and not has_stock:
                    if total_holdings_count >= 3:
                        pass
                    else:
                        invest_ratio = 0.33 if total_holdings_count == 0 else (0.50 if total_holdings_count == 1 else 0.99)
                        buy_qty = 0
                        
                        if region == "KR":
                            buy_qty = int((cash_krw * invest_ratio) / current_price)
                            if buy_qty > 0:
                                if self.fetcher.send_kr_order(symbol, "buy", buy_qty):
                                    # [수정] DB에 매수 기록 저장
                                    self.db.save_trade(symbol, "BUY", current_price, buy_qty)
                                    self.db.log(f"🚀 [매수] {symbol} {buy_qty}주 (@{current_price})")
                        else:
                            buy_qty = int((cash_usd * invest_ratio) / current_price)
                            if buy_qty > 0:
                                if self.fetcher.send_us_order(excd, symbol, "buy", buy_qty, current_price):
                                    # [수정] DB에 매수 기록 저장
                                    self.db.save_trade(symbol, "BUY", current_price, buy_qty)
                                    self.db.log(f"🚀 [매수] {symbol} {buy_qty}주 (@${current_price})")
                        
                        if buy_qty > 0:
                            self.trade_state[symbol] = {'tp1_done': False, 'tp2_done': False}

                # --- [매도 로직] ---
                if has_stock and my_info:
                    if symbol not in self.trade_state:
                        self.trade_state[symbol] = {'tp1_done': False, 'tp2_done': False}
                    
                    state = self.trade_state[symbol]
                    avg_price = my_info['avg_price']
                    current_qty = my_info['qty']
                    profit_rate = (current_price - avg_price) / avg_price
                    
                    sell_qty = 0
                    sell_type = ""
                    
                    # 3% 익절
                    if profit_rate >= 0.03 and not state['tp1_done']:
                        sell_qty = max(1, int(current_qty * 0.5))
                        sell_type = "PROFIT_3%"
                        self.trade_state[symbol]['tp1_done'] = True
                    
                    # 5% 익절
                    elif profit_rate >= 0.05 and not state['tp2_done']:
                        sell_qty = max(1, int(current_qty * 0.5))
                        sell_type = "PROFIT_5%"
                        self.trade_state[symbol]['tp2_done'] = True
                        
                    # 손절 (-4%)
                    elif profit_rate <= -0.04:
                        sell_qty = current_qty
                        sell_type = "CUT_LOSS"
                        del self.trade_state[symbol]
                    
                    if sell_qty > 0:
                        success = False
                        if region == "KR": success = self.fetcher.send_kr_order(symbol, "sell", sell_qty)
                        else: success = self.fetcher.send_us_order(excd, symbol, "sell", sell_qty, current_price)
                        
                        if success:
                            # [수정] DB에 매도 기록 저장
                            self.db.save_trade(symbol, sell_type, current_price, sell_qty, profit_rate * 100)
                            self.db.log(f"📉 [매도] {symbol} ({sell_type}) {sell_qty}주 수익률: {profit_rate*100:.2f}%")

            except Exception as e:
                continue

        # 결과 리포트 (DB 로그로 남김)
        self.db.log(f"📊 [리포트] 총 {count_checked}개 조회 | 매수조건 만족: {count_satisfied}개")

# ==========================================
# 4. 메인 실행
# ==========================================
if __name__ == "__main__":
    db = BotDatabase("trading.db")
    db.log("🤖 봇 시스템 시작 (DB 연동 완료)")
    
    # ⚠️ 보안 주의
    APP_KEY = "PS4O81PxUmBuHjfNABTfiuRFP06eYqulanDt"
    APP_SECRET = "mcO3Qtqq+3cSwbFTWNSV4c0NLP3tdJQ2ABjim1xTZ+BfJt+gL+oKkAUhuXczqr7L2lTdP4Da3T8Dk+O9STBMfUVZXmHsNVqp2V5KCGrkfuF9MNyx8s2sJJ9wONda50V3Y0Vapp3q86RL3aeY33ec4yRnrsO15EITPZN3cejDGSuFO8F3O6w="
    ACCOUNT_NO = "43522038-01" 
    MODE = "real" 

    TARGET_STOCKS = [
        {"region": "KR", "symbol": "005930"}, 
        {"region": "KR", "symbol": "000660"}, 
        {'region': 'KR', 'symbol': '195990'}, {'region': 'KR', 'symbol': '099440'}, {'region': 'KR', 'symbol': '459550'}, {'region': 'KR', 'symbol': '068330'}, {'region': 'KR', 'symbol': '033170'}, {'region': 'KR', 'symbol': '359090'}, {'region': 'KR', 'symbol': '004060'}, {'region': 'KR', 'symbol': '005930'}, {'region': 'KR', 'symbol': '318060'}, {'region': 'KR', 'symbol': '486990'}, {'region': 'KR', 'symbol': '049630'}, {'region': 'KR', 'symbol': '424870'}, {'region': 'KR', 'symbol': '221800'}, {'region': 'KR', 'symbol': '317850'}, {'region': 'KR', 'symbol': '125490'}, {'region': 'KR', 'symbol': '044960'}, {'region': 'KR', 'symbol': '006340'}, {'region': 'KR', 'symbol': '151910'}, {'region': 'KR', 'symbol': '015760'}, {'region': 'KR', 'symbol': '191410'}, {'region': 'KR', 'symbol': '041930'}, {'region': 'KR', 'symbol': '450140'}, {'region': 'KR', 'symbol': '090710'}, {'region': 'KR', 'symbol': '003720'}, {'region': 'KR', 'symbol': '098460'}, {'region': 'KR', 'symbol': '081180'}, {'region': 'KR', 'symbol': '488900'}, {'region': 'KR', 'symbol': '007660'}, {'region': 'KR', 'symbol': '114450'}, {'region': 'KR', 'symbol': '010170'}, {'region': 'KR', 'symbol': '102280'}, {'region': 'KR', 'symbol': '092200'}, {'region': 'KR', 'symbol': '376900'}, {'region': 'KR', 'symbol': '299170'}, {'region': 'KR', 'symbol': '154030'}, {'region': 'KR', 'symbol': '062970'}, {'region': 'KR', 'symbol': '014970'}, {'region': 'KR', 'symbol': '365330'}, {'region': 'KR', 'symbol': '005690'}, {'region': 'KR', 'symbol': '030530'}, {'region': 'KR', 'symbol': '255220'}, {'region': 'KR', 'symbol': '476060'}, {'region': 'KR', 'symbol': '250060'}, {'region': 'KR', 'symbol': '0015G0'}, {'region': 'KR', 'symbol': '177900'}, {'region': 'KR', 'symbol': '000660'}, {'region': 'KR', 'symbol': '058470'}, {'region': 'KR', 'symbol': '001440'}, {'region': 'KR', 'symbol': '456160'}, {'region': 'KR', 'symbol': '032820'}, {'region': 'KR', 'symbol': '101400'}, {'region': 'KR', 'symbol': '353200'}, {'region': 'KR', 'symbol': '494120'}, {'region': 'KR', 'symbol': '244920'}, {'region': 'KR', 'symbol': '488280'}, {'region': 'KR', 'symbol': '034020'}, {'region': 'KR', 'symbol': '256840'}, {'region': 'KR', 'symbol': '067000'}, {'region': 'KR', 'symbol': '006800'}, {'region': 'KR', 'symbol': '084670'}, {'region': 'KR', 'symbol': '004410'}, {'region': 'KR', 'symbol': '347850'}, {'region': 'KR', 'symbol': '005935'}, {'region': 'KR', 'symbol': '027040'}, {'region': 'KR', 'symbol': '001510'}, {'region': 'KR', 'symbol': '093370'}, {'region': 'KR', 'symbol': '058730'}, {'region': 'KR', 'symbol': '042670'}, {'region': 'KR', 'symbol': '033340'}, {'region': 'KR', 'symbol': '204620'}, {'region': 'KR', 'symbol': '031330'}, {'region': 'KR', 'symbol': '085310'}, {'region': 'KR', 'symbol': '010140'}, {'region': 'KR', 'symbol': '317240'}, {'region': 'KR', 'symbol': '468530'}, {'region': 'KR', 'symbol': '058610'}, {'region': 'KR', 'symbol': '104620'}, {'region': 'KR', 'symbol': '125210'}, {'region': 'KR', 'symbol': '0091W0'},
        {"region": "US", "symbol": "NVDA", "excd": "NAS"},
        {"region": "US", "symbol": "ONDS", "excd": "NAS"},
        {"region": "US", "symbol": "OPEN", "excd": "NAS"},
        {"region": "US", "symbol": "PLUG", "excd": "NAS"},
        {"region": "US", "symbol": "NVO", "excd": "NYS"},
        {"region": "US", "symbol": "TSLA", "excd": "NAS"},
        {"region": "US", "symbol": "BBAI", "excd": "NYS"},
        {"region": "US", "symbol": "GRAB", "excd": "NAS"},
        {"region": "US", "symbol": "GOOGL", "excd": "NAS"},
        {"region": "US", "symbol": "OSCR", "excd": "NYS"},
        {"region": "US", "symbol": "INTC", "excd": "NAS"},
        {"region": "US", "symbol": "BMNR", "excd": "AMS"},
        {"region": "US", "symbol": "SOFI", "excd": "NAS"},
        {"region": "US", "symbol": "NIO", "excd": "NYS"},
        {"region": "US", "symbol": "MARA", "excd": "NAS"},
        {"region": "US", "symbol": "CIFR", "excd": "NAS"},
        {"region": "US", "symbol": "GOOG", "excd": "NAS"},
        {"region": "US", "symbol": "PLTR", "excd": "NAS"},
        {"region": "US", "symbol": "DNN", "excd": "AMS"},
        {"region": "US", "symbol": "PFE", "excd": "NYS"},
        {"region": "US", "symbol": "IREN", "excd": "NAS"},
        {"region": "US", "symbol": "AMD", "excd": "NAS"},
        {"region": "US", "symbol": "AAPL", "excd": "NAS"},
        {"region": "US", "symbol": "KVUE", "excd": "NYS"},
        {"region": "US", "symbol": "WULF", "excd": "NAS"},
        {"region": "US", "symbol": "AAL", "excd": "NAS"},
        {"region": "US", "symbol": "AMZN", "excd": "NAS"},
        {"region": "US", "symbol": "CLSK", "excd": "NAS"},
        {"region": "US", "symbol": "RGTI", "excd": "NAS"},
        {"region": "US", "symbol": "F", "excd": "NYS"},
        {"region": "US", "symbol": "NOK", "excd": "NYS"},
        {"region": "US", "symbol": "ABEV", "excd": "NYS"},
        {"region": "US", "symbol": "AVGO", "excd": "NAS"},
        {"region": "US", "symbol": "ORCL", "excd": "NYS"},
        {"region": "US", "symbol": "APLD", "excd": "NAS"},
        {"region": "US", "symbol": "BTG", "excd": "AMS"},
        {"region": "US", "symbol": "T", "excd": "NYS"},
        {"region": "US", "symbol": "RIG", "excd": "NYS"},
        {"region": "US", "symbol": "SNAP", "excd": "NYS"},
        {"region": "US", "symbol": "NFLX", "excd": "NAS"},
        {"region": "US", "symbol": "MU", "excd": "NAS"},
        {"region": "US", "symbol": "RIVN", "excd": "NAS"},
        {"region": "US", "symbol": "HOOD", "excd": "NAS"},
        {"region": "US", "symbol": "BAC", "excd": "NYS"},
        {"region": "US", "symbol": "QBTS", "excd": "NYS"},
        {"region": "US", "symbol": "ACHR", "excd": "NYS"},
        {"region": "US", "symbol": "SOUN", "excd": "NAS"},
        {"region": "US", "symbol": "MSFT", "excd": "NAS"},
        {"region": "US", "symbol": "MRK", "excd": "NYS"},
        {"region": "US", "symbol": "BTE", "excd": "NYS"},
        {"region": "US", "symbol": "PCG", "excd": "NYS"},
        {"region": "US", "symbol": "BABA", "excd": "NYS"},
        {"region": "US", "symbol": "VZ", "excd": "NYS"},
        {"region": "US", "symbol": "RKLB", "excd": "NAS"},
        {"region": "US", "symbol": "B", "excd": "NYS"},
        {"region": "US", "symbol": "NU", "excd": "NYS"},
        {"region": "US", "symbol": "SMR", "excd": "NYS"},
        {"region": "US", "symbol": "CMCSA", "excd": "NAS"},
        {"region": "US", "symbol": "QUBT", "excd": "NAS"},
        {"region": "US", "symbol": "NBIS", "excd": "NAS"},
        {"region": "US", "symbol": "SLB", "excd": "NYS"},
        {"region": "US", "symbol": "HIMS", "excd": "NYS"},
        {"region": "US", "symbol": "SMCI", "excd": "NAS"},
        {"region": "US", "symbol": "BMY", "excd": "NYS"},
        {"region": "US", "symbol": "CRWV", "excd": "NAS"},
        {"region": "US", "symbol": "META", "excd": "NAS"},
        {"region": "US", "symbol": "KEY", "excd": "NYS"},
        {"region": "US", "symbol": "BULL", "excd": "NAS"},
        {"region": "US", "symbol": "CRCL", "excd": "NYS"},
        {"region": "US", "symbol": "IONQ", "excd": "NYS"},
        {"region": "US", "symbol": "RIOT", "excd": "NAS"},
        {"region": "US", "symbol": "MSTR", "excd": "NAS"},
        {"region": "US", "symbol": "VALE", "excd": "NYS"},
        {"region": "US", "symbol": "WBD", "excd": "NAS"},
        {"region": "US", "symbol": "UBER", "excd": "NYS"},
        {"region": "US", "symbol": "COMP", "excd": "NYS"},
        {"region": "US", "symbol": "FOLD", "excd": "NAS"},
        {"region": "US", "symbol": "SNDK", "excd": "NAS"},
        {"region": "US", "symbol": "KO", "excd": "NYS"},
        {"region": "US", "symbol": "RXRX", "excd": "NAS"},
        {"region": "US", "symbol": "CNC", "excd": "NYS"},
        {"region": "US", "symbol": "CLF", "excd": "NYS"},
        {"region": "US", "symbol": "AMRZ", "excd": "NYS"},
        {"region": "US", "symbol": "BE", "excd": "NYS"},
        {"region": "US", "symbol": "EOSE", "excd": "NAS"},
        {"region": "US", "symbol": "BBWI", "excd": "NYS"},
        {"region": "US", "symbol": "EXAS", "excd": "NAS"},
        {"region": "US", "symbol": "QS", "excd": "NYS"},
        {"region": "US", "symbol": "YMM", "excd": "NYS"},
        {"region": "US", "symbol": "ET", "excd": "NYS"},
        {"region": "US", "symbol": "AGNC", "excd": "NAS"},
        {"region": "US", "symbol": "AMCR", "excd": "NYS"},
        {"region": "US", "symbol": "TSM", "excd": "NYS"},
        {"region": "US", "symbol": "HBAN", "excd": "NAS"},
        {"region": "US", "symbol": "VG", "excd": "NYS"},
        {"region": "US", "symbol": "RKT", "excd": "NYS"},
        {"region": "US", "symbol": "WMT", "excd": "NYS"},
        {"region": "US", "symbol": "BBD", "excd": "NYS"}, # 수정됨
        {"region": "US", "symbol": "WRD", "excd": "NAS"},
        {"region": "US", "symbol": "CMG", "excd": "NYS"}  # 수정됨
    ]
    
    AI_PARAMS = {'lookback': 120, 'band_pct': 0.01}
    
    try:
        fetcher = KisDataFetcher(APP_KEY, APP_SECRET, ACCOUNT_NO, mode=MODE)
        # [수정] db 객체 전달
        bot = GlobalRealTimeTrader(fetcher, TARGET_STOCKS, AI_PARAMS, db)
        
        while True:
            try:
                bot.run_check()
                time.sleep(180) # 3분 대기
            except Exception as e:
                db.log(f"⚠️ 에러 발생: {e}")
                time.sleep(10)
                
    except KeyboardInterrupt:
        print("\n🛑 프로그램 종료")

[2025-11-25 13:11:38] 🤖 봇 시스템 시작 (DB 연동 완료)
[2025-11-25 13:11:38] ⏰ 감시 시작 (보유 종목 스캔 중...)
⚠️ get_us_buyable_cash 금액 파싱 실패: 'ovrs_ord_psbl_amt' {'ctx_area_fk200': '                                                                                                                                                                                                        ', 'ctx_area_nk200': '                                                                                                                                                                                                        ', 'output1': [], 'output2': {'frcr_pchs_amt1': '0.00000', 'ovrs_rlzt_pfls_amt': '0.00000', 'ovrs_tot_pfls': '0.00000', 'rlzt_erng_rt': '0.00000000', 'tot_evlu_pfls_amt': '0.00000000', 'tot_pftrt': '0.00000000', 'frcr_buy_amt_smtl1': '0.000000', 'ovrs_rlzt_pfls_amt2': '0.00000', 'frcr_buy_amt_smtl2': '0.000000'}, 'rt_cd': '0', 'msg_cd': 'KIOK0560', 'msg1': '조회할 내용이 없습니다                                           